### 토크나이저 만들기
* 토크나이저를 사용하는 곳
   - 자연어
   - 시퀀스 데이터 적용 가능

* 토크나이저가 필요한 상황
    - 우리의 데이터에 적용할수 있는 토크나이저가 필요함
    - 예시) 로그 분석
    - 학습 시작/ 학습 중지/ 학습 끝/ 볼륨 조절/...
    - 1단계 : 데이터를 전환(한글 ->영어/ 영어단어가 긴경우(study_start) -> 코드화(A)
        - nltk를 적용한다면?, 'study_start-study_end' -> 'study'. 'start' 'study', 'end'
        - 따로 따로 토크나이징 해야함

### 기본토크나이저 만들기 - 기초
* 공백으로 분리되는 토크나이저 만들기
* 반드시 지켜줘야 할 부분:
    - 최종 리턴은 꼭 리스트 타입으로 해야한다
    - parameter는 data(텍스트)만 사용
    

In [1]:
import os
import re
import pandas as pd

In [2]:
!pip install tokenizers

In [3]:
def basicTokenizer(data : str):
    return data.split(' ')

In [4]:
basicTokenizer('나는 너구리가 아니야. 라쿤이라고!')

['나는', '너구리가', '아니야.', '라쿤이라고!']

### 정규표현식으로 만들기

In [5]:
from nltk.tokenize import RegexpTokenizer

In [6]:
retokenizer = RegexpTokenizer("[\w]+")

In [7]:
sample_data="""
손님 여러분 안녕하십니까? 우리열차는 서울행 고속열차입니다. 오늘도 빠르고 편안한 고속철도를 이용해주셔서 감사합니다.
저희 승무원은 손님 여러분께서 즐겁고 편안하게 여행할 수 있도록 정성을 다하겠습니다. 감사합니다.
"""

In [8]:
print(retokenizer.tokenize(sample_data))

['손님', '여러분', '안녕하십니까', '우리열차는', '서울행', '고속열차입니다', '오늘도', '빠르고', '편안한', '고속철도를', '이용해주셔서', '감사합니다', '저희', '승무원은', '손님', '여러분께서', '즐겁고', '편안하게', '여행할', '수', '있도록', '정성을', '다하겠습니다', '감사합니다']


### 문장단위로 나눌수 있는 정규표현식으로 만들고, 적용해보기

In [9]:
def customRegexp(data):
    pattern='[\,\.!?\n]'
    rs=re.split(pattern, data, maxsplit=0)
    return rs

In [10]:
customRegexp(sample_data)

['',
 '손님 여러분 안녕하십니까',
 ' 우리열차는 서울행 고속열차입니다',
 ' 오늘도 빠르고 편안한 고속철도를 이용해주셔서 감사합니다',
 '',
 '저희 승무원은 손님 여러분께서 즐겁고 편안하게 여행할 수 있도록 정성을 다하겠습니다',
 ' 감사합니다',
 '',
 '']

In [11]:
re_rs = customRegexp(sample_data)

In [12]:
re_rs= [rs_unit.strip() for rs_unit in re_rs if len(rs_unit.strip())>1]

In [13]:
re_rs

['손님 여러분 안녕하십니까',
 '우리열차는 서울행 고속열차입니다',
 '오늘도 빠르고 편안한 고속철도를 이용해주셔서 감사합니다',
 '저희 승무원은 손님 여러분께서 즐겁고 편안하게 여행할 수 있도록 정성을 다하겠습니다',
 '감사합니다']

In [14]:
def customRegexp(data):
    pattern='[\,\.!?\n]'
    rs=re.split(pattern, data, maxsplit=0)
    f_rs = [rs_unit.strip() for rs_unit in rs if len(rs_unit.strip())>1]
    return f_rs

In [15]:
customRegexp(sample_data)

['손님 여러분 안녕하십니까',
 '우리열차는 서울행 고속열차입니다',
 '오늘도 빠르고 편안한 고속철도를 이용해주셔서 감사합니다',
 '저희 승무원은 손님 여러분께서 즐겁고 편안하게 여행할 수 있도록 정성을 다하겠습니다',
 '감사합니다']

### re.split(maxsplit=)
 - 최대 몇번까지 나누눌 것인지 
 - 이값이 만약 0이다. 모든 문자열을 나눠줌
 - 이값이 0보다 크면, 그횟수만큼 난누고, 남은 문자열은 그대로 반환
 - 단 읍수는 않됨

In [16]:
def customRegexp_limit3(data):
    pattern='[\,\.!?\n]'
    rs=re.split(pattern, data, maxsplit=3)
    f_rs = [rs_unit.strip() for rs_unit in rs if len(rs_unit.strip())>1]
    return f_rs

In [17]:
customRegexp_limit3(sample_data)

['손님 여러분 안녕하십니까',
 '우리열차는 서울행 고속열차입니다',
 '오늘도 빠르고 편안한 고속철도를 이용해주셔서 감사합니다.\n저희 승무원은 손님 여러분께서 즐겁고 편안하게 여행할 수 있도록 정성을 다하겠습니다. 감사합니다.']

### 데이터 프레임으로 최종결과를 넣을떄, 


In [18]:
tmp_df= pd.DataFrame(customRegexp_limit3(sample_data))
tmp_df_T= tmp_df.T
tmp_df_T.columns = ['sen1','sen2','sen3']
tmp_df_T


,sen1,sen2,sen3
0,손님 여러분 안녕하십니까,우리열차는 서울행 고속열차입니다,오늘도 빠르고 편안한 고속철도를 이용해주셔서 감사합니다.\n저희 승무원은 손님 여러...


### BPE 토크나이저를 훈련시키기

* 어휘 집합 (vocab.josn)과 병합 우선순위(merge.txt)가 있어야 토큰화가 가능하다

In [19]:
train_df= pd.read_csv('./ratings_train.csv')
test_df=pd.read_csv('./ratings_test.csv')

In [20]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  150000 non-null  int64 
 1   id          150000 non-null  int64 
 2   document    149995 non-null  object
 3   label       150000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 4.6+ MB


In [21]:
train_df.dropna(inplace=True)

In [22]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  50000 non-null  int64 
 1   id          50000 non-null  int64 
 2   document    49997 non-null  object
 3   label       50000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [23]:
test_df.dropna(inplace=True)

In [24]:
train_file_nm='bpe_ratings_train.txt'

In [25]:
text_file=open(train_file_nm, 'w')

corp, stri_train_add= set(), str()

train_series= train_df['document'].drop_duplicates()

In [26]:
## 훈련 데이터로 사용하기 위해서는 str로 변환해야 함
print(train_series[:5])

0                                  아 더빙.. 진짜 짜증나네요 목소리
1                    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                        교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
Name: document, dtype: object


In [27]:
train_str = "\n".join(train_series.tolist())
text_file.write(train_str)
text_file.close()
# 말뭉치
train_list=train_str.split(' ')
train_set=set(train_list)## 말뭉치에는 중복되는 문자와 단어가 없도록함

In [28]:
corp.update(train_set)

In [29]:
corp_tmp, str_train_add_tmp = set(), str()
corp_tmp=set() : str_train_add_tmp= str()

SyntaxError: invalid syntax (3410852924.py, line 2)

In [30]:
test_file_nm ='bpe_ratings_test.txt'
test_file=open(test_file_nm,'w')
test_series= test_df['document'].drop_duplicates()
print(test_series[:5])

0                                                  굳 ㅋ
1                                 GDNTOPCLASSINTHECLUB
2               뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아
3                     지루하지는 않은데 완전 막장임... 돈주고 보기에는....
4    3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??
Name: document, dtype: object


In [31]:
test_str="\n".join(test_series.tolist())
test_file.write(test_str)
test_file.close()

In [32]:
test_list= test_str.split(' ')
test_set= set(test_list)
test_corp=set()
test_corp.update(test_set)

In [33]:
from tokenizers import CharBPETokenizer

In [84]:
### 토크나이저 훈련시키기

vocab= 10000
#train_file_nm
bpe_tokenizer = CharBPETokenizer()
bpe_tokenizer.train(
    files= [train_file_nm,test_file_nm],
    vocab_size=vocab,
    special_tokens = ["<unk>"]
)

In [85]:
bpe_tokenizer.save_model('./mode_data')

['./mode_data/vocab.json', './mode_data/merges.txt']

In [86]:
bpe_tokenizer.save('./mode_data/bpe_tokenizer.json')

### 훈련한 bpe 토크나이저로 토큰화하기

In [37]:
from tokenizers import Tokenizer

In [38]:
bpe_tokenizer_pretrained= Tokenizer.from_file("./mode_data/bpe_tokenizer.json")

In [39]:
global bpe_tokenizer_pretrained

In [40]:
def usingBPETokenizer(data) -> list:
    return bpe_tokenizer_pretrained.encode(data).tokens

In [41]:
sample_str=train_df['document'][0]

In [42]:
usingBPETokenizer(sample_str)

['아</w>', '더빙</w>', '.</w>', '.</w>', '진짜</w>', '짜증나', '네요</w>', '목소리</w>']

### 워드 피스 토크나이저 만들기

In [43]:
from tokenizers import BertWordPieceTokenizer

In [44]:
wordpiece_tokenizer = BertWordPieceTokenizer(lowercase=False)
wordpiece_tokenizer.train(
    files=[train_file_nm,test_file_nm],
    vocab_size=vocab
)

In [45]:
wordpiece_tokenizer.save_model('./mode_data')

['./mode_data/vocab.txt']

In [46]:
wordpiece_tokenizer.save('./mode_data/wp_tokenizer.josn')

### 워드피스 토크나이저 적용하기

In [48]:
wp_tokenizer_pretrained = Tokenizer.from_file('./mode_data/wp_tokenizer.josn')

In [49]:
global wp_tokenizer_pretrained

In [50]:
def usingWordPieceTokenizer(data) -> list:
    return wp_tokenizer_pretrained.encode(data).tokens

In [51]:
usingWordPieceTokenizer(sample_str)

['아', '더빙', '.', '.', '진짜', '짜증나', '##네요', '목소리']

In [52]:
test_df['document'].sample(10)

32766    바라고 원하던 판타지 상상. 드래곤+ 바이킹+ 하늘 그리고 중국풍에 개성이 너무 황...
49899    이게 왜 평점이 높은지 이해불가... 날라리 30대남자에게 철없는 어린마음에 몸까지...
10760                           소녀에 대한 연민과 겉잡을수 없는 집착의 딜레마
22441                                 평점가지고 장난금지... 생애 최악!
22831    사람 사는 곳은 어디나 똑같다. 미국이나 한국이나 시골이나 도시나 부자나 가난한 자...
36569                         초반에 일었던 흥미를 굴곡 있게 변주시키지 못한다.
47993                                               불쌍하다..
4467                    일단 화려한 영상이 수준급,배우들의 연기또한 수준급..굿이다.
15994                                    명작 다시봐도 또 보고싶은 영화
13136    주인공부터 대통령딸, 주변인 모두 멍청한 짓만 골라하다가 수조 수천억원의 우주선을 ...
Name: document, dtype: object

In [53]:
print(usingBPETokenizer(test_df['document'].loc[32766]))
print(usingWordPieceTokenizer(test_df['document'].loc[32766]))

['바', '라고</w>', '원', '하던</w>', '판타지</w>', '상', '상</w>', '.</w>', '드래', '곤</w>', '+</w>', '바이', '킹</w>', '+</w>', '하', '늘</w>', '그리고</w>', '중국', '풍', '에</w>', '개', '성이</w>', '너무</w>', '황', '홀', '하다</w>', '.</w>']
['바라', '##고', '원', '##하던', '판타지', '상상', '.', '드래곤', '+', '바이', '##킹', '+', '하늘', '그리고', '중국', '##풍', '##에', '개성', '##이', '너무', '황', '##홀', '##하다', '.']
